# Functions as first-class objects
Functions, like dictionaries, strings and integers, are first-class objects, meaning they can be created at runtime, passed as arguments to other functions, or returns from other functions

## Higher Order Functions
A function that takes a function as an arg or returns a function. Common HOF:
- map
- filter
- reduce
- apply (deprecated in Python 3)

### Map and Filter
*map*: applies a function to every item in a sequence
*filter*: filter a sequence according to a given function
List comprehensions are more readable versions of `map` and `filter`

In [6]:
def foo(a):
    return a ** 2
mylist = [1, 3, 4, 5]

l1 = list(map(foo, mylist))
l2 = [foo(item) for item in mylist]
print(l1 == l2)

l1 = list(map(foo, filter(lambda x: x % 2 == 0, mylist)))
l2 = [foo(item) for item in mylist if item % 2 == 0]
print(l1 == l2)

True
True


### Reduce
Demoted to a functools function. It applies an operation to a successive list of items, accumulating the result until a single value remains

In [10]:
from functools import reduce

v1 = reduce(lambda x, y: x + y, range(10))

# Better to use the built-in reducer sum
v2 = sum(range(10))
print(v1 == v2)

# Other built-in reducers: any, all

True


## Function annotations
Attach metadata to the parameters of a function declaration and its return values. Annotations can be classes (`str`, `int`, `MyClass`, or strings (`int > 0`)

In [12]:
def clip(text:str, max_len:'int > 0'=80) -> str:
    """
    Return text clipped at the last space before or after max_len
    """
    end = None
    if len(text) > max_len:
        space_before = text.rfind(' ', 0, max_len)
        if space_before >= 0:
            end = space_before
        else:
            space_after = text.rfind(' ', max_len)
            if space_after >= 0:
                end = space_after
    if end is None: # no spaces were found
        end = len(text)
    return text[:end].rstrip()

clip.__annotations__

{'max_len': 'int > 0', 'return': str, 'text': str}

## Packages for functional programming

### The `operator` module
Provides functional equivalents to dozens of arithmetic operations (sum, multiply, etc.)

In [35]:
import operator
[func for func in dir(operator) if not func.startswith('_')]

['abs',
 'add',
 'and_',
 'attrgetter',
 'concat',
 'contains',
 'countOf',
 'delitem',
 'eq',
 'floordiv',
 'ge',
 'getitem',
 'gt',
 'iadd',
 'iand',
 'iconcat',
 'ifloordiv',
 'ilshift',
 'imatmul',
 'imod',
 'imul',
 'index',
 'indexOf',
 'inv',
 'invert',
 'ior',
 'ipow',
 'irshift',
 'is_',
 'is_not',
 'isub',
 'itemgetter',
 'itruediv',
 'ixor',
 'le',
 'length_hint',
 'lshift',
 'lt',
 'matmul',
 'methodcaller',
 'mod',
 'mul',
 'ne',
 'neg',
 'not_',
 'or_',
 'pos',
 'pow',
 'rshift',
 'setitem',
 'sub',
 'truediv',
 'truth',
 'xor']

In [24]:
# itemgetter and attrgetter
# itemgetter(1) == lambda fields: fields[1]
# supports any class that implements the __getitem__ method

from operator import itemgetter, attrgetter
# sort a list of tuples by the value of one field
metro_data = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
]

sorted(metro_data, key=itemgetter(2))


[('Sao Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
 ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
 ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
 ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
 ('Tokyo', 'JP', 36.933, (35.689722, 139.691667))]

In [32]:
# attrgetter: extract object attributes by name
from collections import namedtuple
from operator import attrgetter

LatLong = namedtuple('LatLong', 'lat long')
Metropolis = namedtuple('Metropolis', 'name cc pop coord')

metro_areas = [Metropolis(name, cc, pop, LatLong(lat, long))
               for name, cc, pop, (lat, long) in metro_data]

# traditional way to traverse nested object
print(metro_areas[0].coord.lat)

# define an attrgetter object that will retrieve the name and coord.lat nested attributes
name_lat = attrgetter('name', 'coord.lat')
print(name_lat)

# for each city sorted by the coord.lat nested attribute,
for city in sorted(metro_areas, key=attrgetter('coord.lat')):
    # retrieve only the name and latitude attributes as defined earlier
    print(city)
    print(name_lat(city))

35.689722
operator.attrgetter('name', 'coord.lat')
Metropolis(name='Sao Paulo', cc='BR', pop=19.649, coord=LatLong(lat=-23.547778, long=-46.635833))
('Sao Paulo', -23.547778)
Metropolis(name='Mexico City', cc='MX', pop=20.142, coord=LatLong(lat=19.433333, long=-99.133333))
('Mexico City', 19.433333)
Metropolis(name='Delhi NCR', cc='IN', pop=21.935, coord=LatLong(lat=28.613889, long=77.208889))
('Delhi NCR', 28.613889)
Metropolis(name='Tokyo', cc='JP', pop=36.933, coord=LatLong(lat=35.689722, long=139.691667))
('Tokyo', 35.689722)
Metropolis(name='New York-Newark', cc='US', pop=20.104, coord=LatLong(lat=40.808611, long=-74.020386))
('New York-Newark', 40.808611)


### Freezing arguments with `functools.partial`
Given a function, a partial application will produce a new function with some of the arguments of the original function *fixed*. Useful to adapt a function that takes one or more arguments to an API that expects a callback with fewer arguments.

It takes a callable as its first argument, and an arbitrary number of positional and keyword arguments to bind later

In [39]:
from operator import mul
from functools import partial

# mul takes 2 arguments, but triple will take only 1, and fix the other one to 3
triple = partial(mul, 3)

[triple(item) for item in range(5)]

def normalize(x, subtract, divide):
    return (x - subtract) / divide

# 
single_arg = partial(normalize, subtract=0, divide=2)
print(normalize(10, 0, 2))
print(single_arg(10))

5.0
5.0
